In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import sem
import copy

import dm_agents
import dm_sim_period as simp
import dm_process_results as pr
import env_make_agents as mkt

def print_agents(agents):
    for agent in agents:
        print(agent, type(agent))

def change_agents(agents):
    ZIDPR = dm_agents.ZIDPR
    """
    make a copy of agents at current location and change strategy to ZIDPR
    """
    new_agents = []
    for k, agent in enumerate(agents):
        # change name
        if k < 6:
            name = agent.name
            s1 = name.split('_')
            name = s1[0] + '_' + s1[1] + '_ZIDPR'

            trader_type = agent.type
            payoff = agent.payoff
            money = agent.money
            location = agent.location
            lower_bound = agent.lower_bound
            upper_bound = agent.upper_bound
            #cont_flag = agent.contract_this_period
            # make a ZIDPR agent
            new_agent = dm_agents.ZIDPR(name, trader_type, payoff, money, location, 
                                       lower_bound, upper_bound)

            #new_agent.set_contract_this_period(cont_flag)
            if new_agent.get_type() == "BUYER":
                vals = agent.get_values()
                new_agent.set_values(vals)
            else:
                cos = agent.get_costs()
                new_agent.set_costs(cos)
        else:
            new_agent = copy.deepcopy(agent)
            
        new_agents.append(new_agent)
        
    return new_agents

def change_back_agents(agents):
    
    ZIDPA = dm_agents.ZIDPA
    """Returns agents to type ZIDPA"""
    new_agents = []
    
    for k, agent in enumerate(agents):
        # change name
        name = agent.name
        s1 = name.split('_')
        name = s1[0] + '_' + s1[1] + '_ZIDPA'

        trader_type = agent.type
        payoff = agent.payoff
        money = agent.money
        location = agent.location
        lower_bound = agent.lower_bound
        upper_bound = agent.upper_bound
        cont_flag = agent.contract_this_period
        print('+=+=+= ', cont_flag)
        # make a ZIDPA agent
        new_agent = dm_agents.ZIDPA(name, trader_type, payoff, money, location, 
                                   lower_bound, upper_bound)
        
        new_agent.set_contract_this_period(cont_flag)

        if new_agent.get_type() == "BUYER":
            vals = agent.get_values()
            new_agent.set_values(vals)
        else:
            cos = agent.get_costs()
            new_agent.set_costs(cos)
        
        new_agents.append(new_agent)
        

    return new_agents


def make_event_sim(sim_name, num_periods, num_weeks,
             event_begin, event_end, market, agents,
             num_rounds, grid_size,
             num_traders, num_units,
             lower_bound, upper_bound,
             trader_objects):
    """Runs one complete simulation and returns data in
        effs[treatment][trial].  Causes epidemic event from event_begin to event_end
    """ 
    
    data = {}
    for week in range(num_weeks):
        
        if week == event_begin:
            agents = change_agents(agents)
        if week == event_end:
            agents = change_back_agents(agents)
            
        data[week] = {}
        for agent in agents:
            agent.start(None)
        contracts = []
        sim_grids = []
        sim1 = simp.SimPeriod(sim_name, num_rounds, agents, 
               market, grid_size)
        for period in range(num_periods):
            sim1.run_period()
            print(week, period, agents[8])
            grid = sim1.get_grid()
            sim_grids.append(grid)
            contracts.extend(sim1.get_contracts())
        
        
        data[week]['contracts'] = contracts
        data[week]['grids'] = sim_grids
        
        # process results
        pr1 = pr.ProcessResults(market, sim_name, agents, contracts)
        pr1.calc_efficiency()
        pr1.get_results()
        eff = pr1.get_efficiency()
        type_eff = pr1.get_type_surplus()
        data[week]['eff'] = eff # single item put in list to faciliatate looping through data 
        data[week]['type_effs'] = type_eff
    return data


def make_event_monte_carlo(sim_name, num_trials, num_periods, num_weeks,
                    event_begin, event_end,
                    num_rounds, grid_size,
                    num_traders, num_units,
                    lower_bound, upper_bound,
                    trader_objects):
    """Runs one complete simulation and returns data in
        effs[treatment][trial]
    """ 

    sim_data = {}
    sim_data['parms'] = {'sim_name': sim_name, 'num_traders': num_traders, 'num_units': num_units,
                         'num_weeks': num_weeks, 'num_periods': num_periods, 'num_rounds': num_rounds,
                         'grid_size': grid_size, 'lower_bound':lower_bound, 'upper_bound': upper_bound,
                         'trader_objects': trader_objects}

    for trial in range(num_trials):
        
        agent_maker = mkt.MakeAgents(num_traders, trader_objects, num_units, 
                                grid_size, lower_bound, upper_bound)
        agent_maker.make_agents()
        agent_maker.set_locations(grid_size)
        agents = agent_maker.get_agents()

        # set up market
        agent_maker.make_market(sim_name)
        market = agent_maker.get_market()
        
        sim_data[trial]  = make_event_sim(sim_name, num_periods, num_weeks, 
                                    event_begin, event_end, market, agents,
                                    num_rounds, grid_size,
                                    num_traders, num_units,
                                    lower_bound, upper_bound,
                                    trader_objects)
    return sim_data


In [ ]:

ZIDPA = dm_agents.ZIDPA
ZID = dm_agents.ZIDPR
trader_objects =[(ZIDPA, 20),(ZID, 0)] # run simulation with just ZID agents
sim_name = "ZIDPA with event MONTE-CARLO"
num_trials = 10
num_periods = 7
num_weeks = 100
event_begin = 101
event_end = 105
num_rounds = 5
grid_size = 15
num_traders = 20
num_units = 8
lower_bound = 200 
upper_bound = 600

data_table = make_event_monte_carlo(sim_name, num_trials, num_periods, num_weeks,
                                  event_begin, event_end,
                                  num_rounds, grid_size,
                                  num_traders, num_units,
                                  lower_bound, upper_bound,
                                  trader_objects)
print(sim_name, 'finished.')

In [ ]:
for trial in range(num_trials):
    trial_data = data_table[trial]
    for week in range(num_weeks):
        print(trial, week, trial_data[week]['eff'])
    
    

In [ ]:
import dm_sim as sim
eff_avg, std_error = sim.analyze_eff_data(num_trials, num_weeks, data_table)

In [ ]:
import matplotlib.pyplot as plt

x = range(num_weeks)
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(x, eff_avg, label = 'ZIDA', linestyle = 'dotted', color='red', lw =3)
ax.errorbar(x, eff_avg, yerr=std_error, fmt='.k')

ax.set_xlabel('week', size = 'x-large') 
ax.set_xbound(0, num_weeks)
ax.set_ybound(0, 100)
ax.grid(1)
ax.set_ylabel('efficiency', size = 'x-large') 
ax.set_title(f'ZIDPA and ZIDPR average efficiencies for {num_trials} trials', size = 'x-large')
ax.legend(fontsize='x-large')
